In [1]:
import pandas as pd
import pandasql as ps

In [2]:
pedido = pd.read_csv("../data/pedido.csv")
itemPedido = pd.read_csv("../data/item_pedido.csv")

In [3]:
pedido.head()

,idPedido,idCliente,descSituacao,dtPedido,dtAprovado,dtEnvio,dtEntrega,dtEntregaEstimada
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [4]:
itemPedido.head()

,idPedido,idPedidoItem,idProduto,idVendedor,shipping_limit_date,vlPreco,vlFrete
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [5]:
tb_pedido = ps.sqldf("""
                        select 
                            t1.idpedido,
                            t2.idVendedor,
                            t1.descSituacao,
                            t1.dtPedido,
                            t1.dtAprovado,
                            t1.dtEntrega,
                            t1.dtEntregaEstimada,
                            sum(vlFrete) as totalFrete
                            
                        from pedido as t1
                        
                        left join itemPedido as t2
                        on t1.idpedido = t2.idPedido                        
                        
                        where t1.dtPedido < '2018-01-01'
                        and t1.dtPedido >= '2017-07-01'
                        and t2.idVendedor is not null
                        
                        group by t1.idpedido,
                            t2.idVendedor,
                            t1.descSituacao,
                            t1.dtPedido,
                            t1.dtAprovado,
                            t1.dtEntrega,
                            t1.dtEntregaEstimada
                """)

In [6]:
tb_pedido.head()

,idPedido,idVendedor,descSituacao,dtPedido,dtAprovado,dtEntrega,dtEntregaEstimada,totalFrete
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-20 23:43:48,2017-09-29 00:00:00,13.29
1,00054e8431b9d7675808bcb819fb4a32,7040e82f899a04d1b434b795a43b4617,delivered,2017-12-10 11:53:48,2017-12-10 12:10:31,2017-12-18 22:03:38,2018-01-04 00:00:00,11.85
2,000c3e6612759851cc3cbb4b83257986,218d46b86c1881d022bce9c68a7d4b15,delivered,2017-08-12 10:08:57,2017-08-15 03:35:32,2017-08-19 15:22:17,2017-09-01 00:00:00,13.71
3,000e906b789b55f64edcb1f84030f90d,ea8482cd71df3c1969d7b9473ff13abc,delivered,2017-11-21 18:54:23,2017-11-21 19:09:02,2017-12-09 17:27:23,2017-12-07 00:00:00,11.85
4,0010b2e5201cc5f1ae7e9c6cc8f5bd00,3504c0cb71d7fa48d967e0e4c94d59d9,delivered,2017-09-11 17:39:33,2017-09-11 18:04:37,2017-09-23 13:21:21,2017-09-27 00:00:00,16.60


In [7]:
tb_summary = ps.sqldf(""" select 
                            '2018-01-01' as dtReference,
                            idVendedor,
                            cast(count(distinct case when date(coalesce( dtEntrega, 2018-01-01)) > date(dtEntregaEstimada)
                                then idPedido 
                            end) as double) / 
                            cast(count(distinct case when descSituacao = 'delivered' then idPedido end) as double) 
                                as pctPedidoAtraso,
                            cast(count(distinct case when descSituacao = 'canceled' then idPedido end) as double) /
                                cast( count(distinct idPedido) as double) as pctPedidoCancelado,
                                avg(totalFrete) as avgFrete,
                                max(totalFrete) as maxFrete,
                                min(totalFrete) as minFrete,
                                avg(coalesce(dtEntrega, '2018-01-01') - dtPedido) as mediaDiasEntrega,
                                avg(dtEntregaEstimada - coalesce(dtEntrega, '2018-01-01')) as qtdDiasEntregaPromessa
                                
                        from tb_pedido
                        
                        group by idVendedor
                            
                    
                """)

In [8]:
tb_summary.head(30)

,dtReference,idVendedor,pctPedidoAtraso,pctPedidoCancelado,avgFrete,maxFrete,minFrete,mediaDiasEntrega,qtdDiasEntregaPromessa
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,0.000000,0.000000,21.020000,21.02,21.02,0.000000,0.000000
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,0.071429,0.000000,46.686020,143.80,20.27,0.071429,0.030612
2,2018-01-01,002100f778ceb8431b7a1020ff7ab48f,0.157895,0.000000,15.605897,34.15,4.91,0.205128,0.153846
3,2018-01-01,003554e2dce176b5555353e4f3555ac8,0.000000,0.000000,19.380000,19.38,19.38,0.000000,1.000000
4,2018-01-01,004c9cd9d87a3c30c522c48c4fc07416,0.059524,0.000000,21.820357,94.94,12.33,0.047619,0.095238
5,2018-01-01,00ee68308b45bc5e2660cd833c3f81cc,0.024390,0.000000,22.978049,98.02,8.96,0.121951,0.146341
6,2018-01-01,010543a62bd80aa422851e79a3bc7540,0.000000,0.000000,15.975000,18.71,13.24,0.000000,0.000000
7,2018-01-01,013900e863eace745d3ec7614cab5b1a,0.000000,0.000000,25.993750,47.46,16.16,0.250000,0.250000
8,2018-01-01,014c0679dd340a0e338872e7ec85666a,0.000000,0.000000,55.103333,93.35,21.72,0.333333,0.000000
9,2018-01-01,014d9a685fd57276679edd00e07089e5,0.000000,0.000000,2.660000,2.66,2.66,0.000000,0.000000


In [9]:
pathVendedorEntrega = '../data/fs_vendedor_entrega.csv'

In [10]:
tb_summary.to_csv(pathVendedorEntrega, index = False, encoding = 'utf-8')